In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.impute import SimpleImputer
from fastai.imports import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error

In [2]:
PATH = "data-bluebook-for-bulldozers/"

In [3]:
!ls {PATH}

Data Dictionary.xlsx             TrainAndValid.zip
Machine_Appendix.csv             Valid.7z
Test.csv                         Valid.csv
Train.7z                         Valid.zip
Train.csv                        ValidSolution.csv
TrainAndValid.7z                 median_benchmark.csv
TrainAndValid.csv                random_forest_benchmark_test.csv


In [4]:
data_raw = pd.read_csv(f'{PATH}Train.csv',low_memory=False, 
                     parse_dates=["saledate"] )

Always make a copy of the dataset so that you don't alter the existing one.

In [5]:
data = data_raw.copy()

In [6]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [7]:
display_all(data)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,Track Excavators,NaN,EROPS w AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,EROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,35NX2,35,NX,2,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,35NX2,35,NX,2,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,35NX2,35,NX,2,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,30NX,30,NX,NaN,NaN,M

In [8]:
display_all(data.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125,NaN,NaN,NaN,NaT,NaT,1.91971e+06,909021,1.13925e+06,1.41837e+06,1.63942e+06,2.24271e+06,6.33334e+06
SalePrice,401125,NaN,NaN,NaN,NaT,NaT,31099.7,23036.9,4750,14500,24000,40000,142000
MachineID,401125,NaN,NaN,NaN,NaT,NaT,1.2179e+06,440992,0,1.0887e+06,1.27949e+06,1.46807e+06,2.48633e+06
ModelID,401125,NaN,NaN,NaN,NaT,NaT,6889.7,6221.78,28,3259,4604,8724,37198
datasource,401125,NaN,NaN,NaN,NaT,NaT,134.666,8.96224,121,132,132,136,172
auctioneerID,380989,NaN,NaN,NaN,NaT,NaT,6.55604,16.9768,0,1,2,4,99
YearMade,401125,NaN,NaN,NaN,NaT,NaT,1899.16,291.797,1000,1985,1995,2000,2013
MachineHoursCurrentMeter,142765,NaN,NaN,NaN,NaT,NaT,3457.96,27590.3,0,0,0,3025,2.4833e+06
UsageBand,69639,3,Medium,33985,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17,2011-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data.SalePrice = np.log(data.SalePrice)

# Data Preprocessing

This dataset contains a mix of continuous and categorical variables.

The following method extracts particular date fields from a complete datetime for the purpose of constructing categoricals. You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

In [10]:
data['saleyear'] = data['saledate'].dt.year
data.drop('saledate', axis=1,inplace=True)

In [11]:
data.saleyear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleyear, dtype: int64

In [12]:
type(data['saleyear'][0])

numpy.int64

We have lots of missing values, which we can't pass directly to a random forest. So first we will find how many missing values we have followed by dealing with it.

In [13]:
display_all(data.isnull().sum().sort_index()/len(data))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call train_cats to convert strings to pandas categories.

In [14]:
data.UsageBand.head()

0       Low
1       Low
2      High
3      High
4    Medium
Name: UsageBand, dtype: object

In [15]:
data.UsageBand.unique()

array(['Low', 'High', 'Medium', nan], dtype=object)

In [16]:
data.UsageBand = pd.Categorical(data.UsageBand, categories=['High', 'Medium', 'Low'], ordered=True).codes

By default pandas takes nan as -1

In [17]:
data.UsageBand.unique()

array([ 2,  0,  1, -1])

First categorical column has been converted. So we will try with others as well. 

But the problem is that most the column has lots of unique value and doing it manually is a tedious job. We have 
to find a way to deal with missing values and as well as categorial columns. Both cannot be handled simutaneouly. 


------------

Using <b>data.info() </b>we can find which of the columns are object. Object dtypes are strings values and belong to the categorical dataset



In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null float64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   401125 non-null int8
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                       401125 non-null object
ProductGroup                

In [19]:
data.Steering_Controls.unique()

array(['Conventional', nan, 'Command Control', 'Four Wheel Standard',
       'Wheel', 'No'], dtype=object)

In [20]:
categories = []
for i in data.Steering_Controls.unique():
    categories.append(i)

In [21]:
categories.remove(np.nan)

In [22]:
categories

['Conventional', 'Command Control', 'Four Wheel Standard', 'Wheel', 'No']

We will make a function that will first iterate based on the object dtypes--because of the reason mentioned above. Then it will categorise the columns based upon the above intuition. Since not all the columns will have missing values we will use try and except to get away with the error. Then we will drop the dependent column after making the new one. 

In the end we will replace -1 with np.nan because pandas take nan as -1

In [23]:
def assigning_categorical(data, column):
    categories = []
    for i in data[column].unique():
        categories.append(i)
    try:    
        categories.remove(np.nan)
    except Exception as e:
        pass
    data[column+'_new'] = pd.Categorical(data[column], categories=categories, ordered=True).codes
    data.drop(f'{column}', axis=1, inplace = True)
    data.replace(-1, np.nan, inplace=True)
    

In [24]:
assigning_categorical(data, 'Steering_Controls')

In [25]:
data.Steering_Controls_new.unique()

array([ 0., nan,  1.,  2.,  3.,  4.])

In [26]:
for i in data.columns:
    if data[i].dtype == 'O':
        try:
            assigning_categorical(data, i)
        except Exception as e:
            pass

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
SalesID                        401125 non-null int64
SalePrice                      401125 non-null float64
MachineID                      401125 non-null int64
ModelID                        401125 non-null int64
datasource                     401125 non-null int64
auctioneerID                   380989 non-null float64
YearMade                       401125 non-null int64
MachineHoursCurrentMeter       142765 non-null float64
UsageBand                      69639 non-null float64
saleyear                       401125 non-null int64
Steering_Controls_new          69369 non-null float64
fiModelDesc_new                401125 non-null int16
fiBaseModel_new                401125 non-null int16
fiSecondaryDesc_new            263934 non-null float64
fiModelSeries_new              56908 non-null float64
fiModelDescriptor_new          71919 non-null float64
ProductSize_new              

In [28]:
data.isnull().sum()/len(data)

SalesID                        0.000000
SalePrice                      0.000000
MachineID                      0.000000
ModelID                        0.000000
datasource                     0.000000
auctioneerID                   0.050199
YearMade                       0.000000
MachineHoursCurrentMeter       0.644089
UsageBand                      0.826391
saleyear                       0.000000
Steering_Controls_new          0.827064
fiModelDesc_new                0.000000
fiBaseModel_new                0.000000
fiSecondaryDesc_new            0.342016
fiModelSeries_new              0.858129
fiModelDescriptor_new          0.820707
ProductSize_new                0.525460
fiProductClassDesc_new         0.000000
state_new                      0.000000
ProductGroup_new               0.000000
ProductGroupDesc_new           0.000000
Drive_System_new               0.739829
Enclosure_new                  0.000810
Forks_new                      0.521154
Pad_Type_new                   0.802720


In [29]:
data.isna().sum()/len(data)

SalesID                        0.000000
SalePrice                      0.000000
MachineID                      0.000000
ModelID                        0.000000
datasource                     0.000000
auctioneerID                   0.050199
YearMade                       0.000000
MachineHoursCurrentMeter       0.644089
UsageBand                      0.826391
saleyear                       0.000000
Steering_Controls_new          0.827064
fiModelDesc_new                0.000000
fiBaseModel_new                0.000000
fiSecondaryDesc_new            0.342016
fiModelSeries_new              0.858129
fiModelDescriptor_new          0.820707
ProductSize_new                0.525460
fiProductClassDesc_new         0.000000
state_new                      0.000000
ProductGroup_new               0.000000
ProductGroupDesc_new           0.000000
Drive_System_new               0.739829
Enclosure_new                  0.000810
Forks_new                      0.521154
Pad_Type_new                   0.802720


We'll replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable. 

We will fill the missing values with median. 

In [30]:
for i in data.columns:
    data[i] = data[i].fillna(data[i].median())

In [31]:
data.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saleyear,...,Track_Type_new,Undercarriage_Pad_Width_new,Stick_Length_new,Thumb_new,Pattern_Changer_new,Grouser_Type_new,Backhoe_Mounting_new,Blade_Type_new,Travel_Controls_new,Differential_Type_new
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,2.0,2006,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,2.0,2004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,0.0,2004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,0.0,2011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,1.0,2009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [32]:
data.isna().sum()/len(data)

SalesID                        0.0
SalePrice                      0.0
MachineID                      0.0
ModelID                        0.0
datasource                     0.0
auctioneerID                   0.0
YearMade                       0.0
MachineHoursCurrentMeter       0.0
UsageBand                      0.0
saleyear                       0.0
Steering_Controls_new          0.0
fiModelDesc_new                0.0
fiBaseModel_new                0.0
fiSecondaryDesc_new            0.0
fiModelSeries_new              0.0
fiModelDescriptor_new          0.0
ProductSize_new                0.0
fiProductClassDesc_new         0.0
state_new                      0.0
ProductGroup_new               0.0
ProductGroupDesc_new           0.0
Drive_System_new               0.0
Enclosure_new                  0.0
Forks_new                      0.0
Pad_Type_new                   0.0
Ride_Control_new               0.0
Stick_new                      0.0
Transmission_new               0.0
Turbocharged_new    

In [33]:
forr = RandomForestRegressor(n_jobs=-1)

In [34]:
forr.fit(data.drop('SalePrice', axis=1), data.SalePrice)

/Users/nielspace/Documents/MacOS ENVI/aiENVI/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [35]:
forr.score(data.drop('SalePrice', axis=1), data.SalePrice)

0.9817335452460855